In [ ]:
import boto3
import datetime
import json
from dateutil.tz import tzlocal

import get_test_details
import query_experiment_details
import aggregate_statistics
import plot

stepfunctions = boto3.client('stepfunctions')
cloudwatch_logs = boto3.client('logs')
cloudformation = boto3.client('cloudformation')
cloudwatch = boto3.client('cloudwatch')

In [ ]:
test_params = []

In [ ]:
test_params.extend([
    {'execution_arn': '' }
])

In [ ]:
test_details = get_test_details.get_test_details(test_params, stepfunctions, cloudformation)

In [ ]:
(producer_stats, consumer_stats) = query_experiment_details.query_cw_logs(test_details, cloudwatch_logs)

In [ ]:
partitions = {
    'ignore_keys': [ 'topic_id', 'cluster_id', 'test_id', 'cluster_name', 'num_producers', ],
    'title_keys': [ 'broker_storage', 'in_cluster_encryption', 'duration_sec', 'kafka_version', ],
    'row_keys': [ 'client_props.producer', 'client_props.consumer', ],
    'column_keys': [ 'consumer_groups.num_groups' ],
    'metric_color_keys': [ 'brokers', 'num_partitions', ],
}

filter_fn = lambda x: True
filter_agg_fn = lambda x: x['test_results']['sent_div_requested_mb_per_sec']>=0.995

filtered_producer_stats = list(filter(filter_fn, producer_stats))
filtered_consumer_stats = filter(filter_fn, consumer_stats)

(producer_aggregated_stats, consumer_aggregated_stats, combined_stats) = aggregate_statistics.aggregate_cw_logs(filtered_producer_stats, filtered_consumer_stats, partitions)
filtered_producer_aggregated_stats = list(filter(filter_agg_fn, producer_aggregated_stats))


In [ ]:
plot.plot_measurements(filtered_producer_aggregated_stats, ['latency_ms_p50_mean', 'latency_ms_p95_mean', 'latency_ms_p99_mean', 'latency_ms_p999_mean' ], 'producer put latency (ms)', **partitions, ymax=350)
plot.plot_measurements(filtered_producer_aggregated_stats, ['latency_ms_p50_stdev', 'latency_ms_p95_stdev', 'latency_ms_p99_stdev', 'latency_ms_p999_stdev' ], 'producer put latency stdev (ms)', **partitions, )

In [ ]:
plot.plot_measurements(producer_aggregated_stats, ['sent_div_requested_mb_per_sec'], 'sent / requested throughput (mb/sec)', **partitions, ymin=0.990, ymax=1.01)
plot.plot_measurements(producer_aggregated_stats, ['actual_duration_div_requested_duration_sec_max'], 'actual test duration/requested test duration (ratio)', **partitions)
plot.plot_measurements(producer_aggregated_stats, ['num_tests'], 'number of tests (count)', **partitions)